In [ ]:
"""
Team Nova:
Arpit Palo
Arun Kumar
Nisarg Gupta
Omkar Kanade
"""

# PyTorch: Multiple Linear Regression + significance of coefficients
# Code to perform linear regression on temperature vs greenhouse gases data and to get significance of the coefficients

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
df = pd.read_csv("/content/merged_temp_ghg.csv", engine="pyarrow", index_col=0)

In [ ]:
df_cleaned = df.replace(np.nan, 0)

In [ ]:
df_cleaned.isna().sum()

CountryCode          0
Month                0
Year                 0
tmpf                 0
dwpf                 0
mslp                 0
sknt                 0
feel                 0
relh                 0
ice_accretion_6hr    0
p01i                 0
Methane Levels       0
CO2 Levels           0
NO Levels            0
dtype: int64

In [ ]:
df_cleaned.columns

Index(['CountryCode', 'Month', 'Year', 'tmpf', 'dwpf', 'mslp', 'sknt', 'feel',
       'relh', 'ice_accretion_6hr', 'p01i', 'Methane Levels', 'CO2 Levels',
       'NO Levels'],
      dtype='object')

In [ ]:
numerical_cols = ['tmpf', 'dwpf', 'mslp', 'sknt', 'feel',
       'relh', 'ice_accretion_6hr', 'p01i', 'Methane Levels', 'CO2 Levels',
       'NO Levels']

df_cleaned[numerical_cols] = (df_cleaned[numerical_cols] - df_cleaned[numerical_cols].mean()) / df_cleaned[numerical_cols].std()

In [ ]:
X_vals = df_cleaned[['dwpf', 'mslp', 'sknt', 'feel',
       'relh', 'ice_accretion_6hr', 'p01i', 'Methane Levels', 'CO2 Levels',
       'NO Levels']].values

y_vals = df_cleaned[['tmpf']].values

In [ ]:
import statsmodels.api as sm
# X = sm.add_constant(X_vals)

# Create and fit the linear regression model
model = sm.OLS(y_vals, X_vals)
results = model.fit()

# Print the summary of the model
print(results.summary())







                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.735
Model:                            OLS   Adj. R-squared (uncentered):              0.735
Method:                 Least Squares   F-statistic:                          2.383e+04
Date:                Sun, 14 May 2023   Prob (F-statistic):                        0.00
Time:                        02:49:37   Log-Likelihood:                         -64723.
No. Observations:               85801   AIC:                                  1.295e+05
Df Residuals:                   85791   BIC:                                  1.296e+05
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [ ]:
# Define the linear regression model
class LinearRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
    
    def forward(self, x):
        return self.linear(x)

In [ ]:
X = torch.tensor(X_vals)
y = torch.tensor(y_vals)

input_size = X.shape[1]
output_size = y.shape[1]
model = LinearRegression(input_size, output_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.05)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X.float())
    loss = criterion(outputs, y.float())
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
# Get the coefficients of beta for the fitted linear regression
coefficients = model.linear.weight.data
intercept = model.linear.bias.data

print("Coefficients:")
print(coefficients)
print("Intercept:")
print(intercept)


Coefficients:
tensor([[ 8.5282e-01,  4.6394e-02, -3.9263e-02,  2.0962e-05,  7.8934e-05,
         -2.3578e-03, -2.4759e-03,  2.1385e-02,  1.2582e-02,  3.9765e-03]])
Intercept:
tensor([6.9203e-10])


In [ ]:
residuals = y.float() - outputs

# Calculate RSS
rss = torch.sum(residuals ** 2).item()

print("Residuals:")
print(residuals)
print("RSS:")
print(rss)

Residuals:
tensor([[-0.1326],
        [ 0.0763],
        [ 0.2565],
        ...,
        [ 0.7398],
        [ 0.3861],
        [ 0.1616]], grad_fn=<SubBackward0>)
RSS:
22727.091796875


In [ ]:
rmse = rss/len(y)
print(rmse)

0.26488143258091396


In [ ]:
from scipy.stats import t

n = len(X)
p = X.shape[1] - 1
df = n - p - 1

residuals = y.float() - outputs
# # Calculate the residual sum of squares (RSS)
rss = torch.sum(residuals**2)

# # Calculate the residual standard error (RSE)
rse = torch.sqrt(rss / df)

# # Calculate the variance-covariance matrix
cov_matrix = rse**2 * torch.inverse(torch.matmul(X.T, X))

# # Get the standard errors of the beta coefficients
se_betas = torch.sqrt(torch.diag(cov_matrix)[:, None])

# # Calculate the t-values
t_values = torch.argmax(coefficients / se_betas)

# # Calculate the p-values
p_values = 2 * (1 - t.cdf(torch.abs(t_values), df))

In [ ]:
p_values

0.0

In [ ]:
l = coefficients.tolist()[0]

In [ ]:
tstat = np.array(coefficients.tolist()[0])/np.sqrt(rmse)

In [ ]:
tstat

array([ 1.65703669e+00,  9.01434970e-02, -7.62884988e-02,  4.07291691e-05,
        1.53369874e-04, -4.58125594e-03, -4.81060799e-03,  4.15510537e-02,
        2.44467540e-02,  7.72646129e-03])

In [ ]:
t = np.array(X)
tvar = np.var(t, axis = 0)
tvar


array([0.99998835, 0.99998835, 0.99998835, 0.99998835, 0.99998835,
       0.99998835, 0.99998835, 0.99998835, 0.99998835, 0.99998835])

In [ ]:
from scipy.stats import t

In [ ]:
for ele in tstat.tolist():
  cd = 1-t.cdf(abs(ele), df)
  print(round(cd, 3))

0.049
0.464
0.47
0.5
0.5
0.498
0.498
0.483
0.49
0.497


In [ ]:
for ele in tstat:
  print(round(ele,2))

1.66
0.09
-0.08
0.0
0.0
-0.0
-0.0
0.04
0.02
0.01
